##### TP fait en binome avec Gabriel Ballot

In [2]:
# Import all libraries needed
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize
import scipy.sparse
import scipy.sparse.linalg

#n images of p pixels
n = 400;
p = 10304;

In [2]:
def build_matrix_from_faces(folder='att_faces', minidata=False):
    # load images
    # 400 images of size (112, 92)
    M = []
    if minidata is True:
        nb_subjects = 1
    else:
        nb_subjects = 40
    for subject in range(1, nb_subjects + 1
                        ):
        for image in range(1, 11):
            face = plt.imread(folder + '/s' + str(subject)
                              + '/' + str(image) + '.pgm')
            M.append(face.ravel())

    return np.array(M, dtype=float)

def vectorize(W, H):
    return np.concatenate((W.ravel(), H.ravel()))

def unvectorize_M(W_H, M):
    # number of elements in W_H is (n+p)*k where M is of size n x m
    # W has the nk first elements
    # H has the kp last elements
    n, p = M.shape
    k = W_H.shape[0] // (n + p)
    W = W_H[:n * k].reshape((n, k))
    H = W_H[n * k:].reshape((k, p))
    return W, H

# Small data to test the algorithm
M_s = build_matrix_from_faces(folder='att_faces', minidata=True)
def unvectorize(W_H): return unvectorize_M(W_H, M)
k_s = 2

# Full data
M = build_matrix_from_faces(folder='att_faces', minidata=False)
def unvectorize(W_H): return unvectorize_M(W_H, M)
k = 38

# 1- Database

##### There are 10 pictures of the face of 40 different people so in total we have 400 pictures. Each picture is 92x112 pixels, with 256 grey levels per pixel.

# 2- Presentation of the model

### Question 2-1

Show that the objective function is not convex. Calculate its gradient. Is the gradient Lipschitz continuous?

**Non convexity :**

Let  $\;\tilde{f}:x,y \mapsto (1-xy)^2$.  
We have,
\begin{align}
\tilde{f}(0,0)&=1 \\
\tilde{f}(1,1)&=0 \\
\tilde{f}(-1,-1)&=0 \\
\end{align}  
with,
$$(0,0) = \frac{1}{2}(1,1)+\frac{1}{2}(-1,-1)$$
and,
$$\tilde{f}(0,0) > \frac{1}{2}\tilde{f}(1,1)+\frac{1}{2}\tilde{f}(-1,-1)$$
Thus, $\tilde{f}$ is not convexe.

Then, 
$$ \tilde{f}(x, y) = f(0,\dots,0,x,0,\dots,0,y,0\dots,0) $$

This is why f is not convexe.

**Gradient :**

\begin{align}
\frac{\partial f}{\partial W_{a,b}} &= \frac{1}{2np} \sum_{i=1}^n \sum_{l=1}^p 2(M_{i,l}-[WH]_{i,l})\frac{\partial}{\partial W_{a,b}} \Big( -\sum_{j=1}^k W_{i,j}H_{j,l} \Big) \\
&= -\frac{1}{2np} \sum_{i=1}^n \sum_{l=1}^p 2(M_{i,l}-[WH]_{i,l}) \delta_{i,a} \delta_{j,b} H_{b,l} \\
&= -\frac{1}{np} \sum_{l=1}^p (M_{a,l}-[WH]_{a,l}) H_{b,l} \\
&= -\frac{1}{np} [M-WH]_a [H]_b^T \\
\\
\frac{\partial f}{\partial H_{a,b}} &= \frac{1}{2np} \sum_{i=1}^n \sum_{l=1}^p 2(M_{i,l}-[WH]_{i,l})\frac{\partial}{\partial H_{a,b}} \Big( -\sum_{j=1}^k W_{i,j}H_{j,l} \Big) \\
&= -\frac{1}{2np} \sum_{i=1}^n \sum_{l=1}^p 2(M_{i,l}-[WH]_{i,l}) \delta_{l,b} \delta_{j,a} W_{i,a} \\
&= -\frac{1}{np} \sum_{i=1}^n (M_{i,b}-[WH]_{i,b}) W_{i,a} \\
&= -\frac{1}{np} [W^T]_a [M-WH]_b^T  \\
\\
\\
\Delta f&= \frac{1}{np}\begin{pmatrix}
[M-WH]_1 [H]_1^T \\
\vdots \\
[M-WH]_n [H]_k^T\\
[W^T]_1 [M-WH]_1^T \\
\vdots \\
[W^T]_k [M-WH]_n^T \\
\end{pmatrix} \\
\end{align}

**$\Delta f$ is not Lipschitz continuous :**

The gradient is proportionnal to $WHH^T$, It can't be majored by a affine plan.

# 3- Find W when H0 is fixed

### Question 3-1

In [3]:
#Initialization of the optimization problem
W0, S, H0 = scipy.sparse.linalg.svds(M, k);
W0 = np.maximum(0, W0 * np.sqrt(S));
H0 = np.maximum(0,(H0.T * np.sqrt(S)).T);

#Initialization of the optimization problem (small matrix)
W0_s, S_s, H0_s = scipy.sparse.linalg.svds(M_s, k_s);
W0_s = np.maximum(0, W0_s * np.sqrt(S_s));
H0_s = np.maximum(0,(H0_s.T * np.sqrt(S_s)).T);

##### It is clever to use this initialization beacause the svd function gives exactly what we need: M = W0*S*H0 = (W0*sqrt(S)) * (sqrt(S)*H0)

##### Taking H0 as a constant, the gradient will become Lipschitz-continuous 

### Question 3-2

$$ g(W) = \frac{1}{2np}||M-WH_0||_F^2 $$
\begin{align}
g(W+w) &= \frac{1}{2np}||M-WH_0 +wH_0||_F^2 \\
g(W+w) &= \frac{1}{2np}(||M-WH_0||_F^2 + ||wH_0||_F^2 + 2<M-WH_0,wH_0>_F) \\
g(W+w) &= g(W) + 2<(M-WH_0)H_0^T,w>_F +o(||wH_0||_F^2)) \\
\end{align}
So,
$$ \Delta g = 2(M-WH_0)H_0^T $$

### Question 3-3

In [4]:
def g(W):
    X = M - np.dot(W,H0)
    return np.trace(np.dot(X,X.T))/(2*n*p)

def grad_g(W):
    tmp= 2*(M - np.dot(W, H0));
    return np.dot(tmp, np.transpose(H0));

### Question 3-4

Computation of $\text{prox}_{\gamma \mathbb{1}_{\mathbb{R}_+}}$.

\begin{align}
\text{prox}_{\gamma i_{\mathbb{R}_+}}(x) &= {\text{argmin}}_{y\in \mathbb{R}} (\gamma i_{\mathbb{R}_+} + \frac{1}{2}||y-x||^2) \\
&= {\text{argmin}}_{y\in \mathbb{R}^+} (\frac{1}{2}||y-x||^2) \\
&= \text{proj}_{\mathbb{R}^+}(x)
\end{align}

### Question 3-5

We want to minimize $g$ subject to non negativity constraints. This is equivalent to minimize :

$$ g + \iota_{\mathbb{R}_+} $$

because this function is equals to $+\infty$ on $]-\infty,0[$.

Has $g$ has a Lipschitz gradient, we can use the proximal gradient method to compute 

$$min_{W \in \mathbb{R}} \;\; g(W) + \iota_{\mathbb{R}_+}(W) $$

At each step, $W_{k+1}$ is given by

$$ W_{k+1} = \text{prox}_{\gamma_k \iota_{\mathbb{R}_+}}\big( W_k - \gamma_k \nabla g(W_k)\big) $$

with $\gamma_k > 0$ the step size.

In [ ]:
def projected_gradient_method(val_g, grad_g, W0, gamma, N):
    for i in range(N) :
        tmp = W0 - gamma * grad_g(W0)
        W0 = np.maximum(0,tmp)
    return W0, val_g(W0)

### Question 3-6

In [4]:
Ls = np.linalg.norm(np.dot(np.transpose(H0_s), H0_s))
print(Ls)
t = time()
mini = projected_gradient_method(g, grad_g, W0, 1, 100)
print("W_min = ", mini[0])
print("\ng(W_min) = ", mini[1])
print("time : ",time()-t)

NameError: name 'H0_s' is not defined

# 4- Algorithmic refinement for the problem with H0 fixed

### Question 4-1

In [32]:
#Projected gradient method with line search
def line_search_projected_gradient_method(val_g, grad_g, W0, gamma, N):
    for i in range(N) :
        f = lambda gamma : val_g(W0 - gamma * grad_g(W0))
        gamma = minimize_scalar(f).x
        W0 = np.maximum(0, W0 - gamma * grad_g(W0))
    return W0, val_g(W0)

t = time()
mini = line_search_projected_gradient_method(g, grad_g, W0, 1, 100)
print("W_min = ", mini[0])
print("\ng(W_min) = ", mini[1])
print("time : ",time()-t)

[[1 2]
 [3 4]]
3


### Question 4-2

The second algorithm gives a better minimum (instead of 534). Nevertheless, it takes a lot more time (166 seconds instead of 6.5 seconds).
We could use an approximate line search to be quicker.